In [2]:
%pip install pip install -q -U google-genai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement install (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Isaác Mauricio\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for install


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Isaác Mauricio\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from typing import Optional, Literal
from pydantic import BaseModel, Field
from google import genai
from google.genai import types
import os
from dotenv import load_dotenv
import logging

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

# Cargamos nuestra API_KEY de Gemini
load_dotenv()
API_KEY = os.getenv("API_KEY")

# Se define la variable global del modelo a utilizar en las requests
model="gemini-2.0-flash"


In [15]:

# --------------------------------------------------------------
# Step 1: Definimos las estruturas de datos para enrutar al modelo
# --------------------------------------------------------------
conf_score_descrp = "Puntuación de confianza del 0 al 1"

class PrimaryRequestType(BaseModel):
    """ Llamada enrutadora a la LLM: Contiene qué tipo de solitud primaria está siendo realizada"""

    request_type: Literal["calendario", "extrac_inf","otro"] = Field(
        description="Tipo de solicitud primaria realizada"
    )
    confidence_score: float = Field(description=conf_score_descrp)
    description: str = Field(description="Descripción de la solicitud")

class CalendarRequestType(BaseModel):
    """Llamada enrutadora a la LLM: Contiene qué tipo de solicitud de calendario está siendo realizada"""

    request_type: Literal["nuevo_evento", "modif_evento", "otro"] = Field(
        description="Tipo de solicitud de calendario realizada"
    )
    confidence_score: float = Field(description=conf_score_descrp)
    description: str = Field(description="Cleaned description of the request")


class NewEventDetails(BaseModel):
    """Detalles para la creación de un nuevo evento"""

    name: str = Field(description="Nombre del evento")
    date: str = Field(description="Fecha y hora del evento. (ISO 8601)")
    duration_minutes: int = Field(description="Duración en minutos")
    participants: list[str] = Field(description="Lista de participantes")


class Change(BaseModel):
    """Detalles a cambiar del evento existente"""

    field: str = Field(description="Campo a cambiar")
    new_value: str = Field(description="Nuevo valor a cambiar")


class ModifyEventDetails(BaseModel):
    """Detalles a modificar un evento existente"""

    event_identifier: str = Field(
        description="Descripción para identificar el evento existente"
    )
    changes: list[Change] = Field(description="Lista de cambios a realizar")
    participants_to_add: list[str] = Field(description="Nuevo participante a añadir")
    participants_to_remove: list[str] = Field(description="Participantes a remover")


class CalendarResponse(BaseModel):
    """Formato de respuesta final"""

    success: bool = Field(description="Si la operación tuvo éxito")
    message: str = Field(description="Mensaje de respuesta amistable al usuario")
    calendar_link: Optional[str] = Field(description="Link de calendario si aplica")


# --------------------------------------------------------------
# Step 2: Define el enrutamiento y las funciones de procesamiento 
# --------------------------------------------------------------

def route_primary_request(user_input: str) -> PrimaryRequestType:
    """ Enruta la llamada a la LLM para determinar el tipo de solicitud"""
    logger.info("Enrutando la solicitud primaria")
    
    client = genai.Client(api_key=API_KEY)
    system_instructions="Determina si la solicitud del usuario está relacionada con los calendarios o con extracción de información"
    response = client.models.generate_content(
        model=model,
        contents=f"{system_instructions}\n\nUser Query: {user_input}",
        config = { 
            "response_mime_type" : 'application/json',
            "response_schema" : PrimaryRequestType
        }        
    )
    result : PrimaryRequestType = response.parsed
    logger.info(f"Solicitud enrutada como: {result.request_type} con una confianza de: {result.confidence_score}")
    return result
    

def route_calendar_request(user_input: str) -> CalendarRequestType:
    """Enruta la llamada a la LLM para determinar el tipo de solicitud de calendario"""
    logger.info("Enrutando solicitud de calendario")

    client = genai.Client(api_key=API_KEY)
    system_instructions="Determina si esta es una solicitud para crear un nuevo evento de calendario o modificar un evento existente"
    response = client.models.generate_content(
        model=model,
        contents=f"{system_instructions}\n\nUser Query: {user_input}",
        config = { 
            "response_mime_type" : 'application/json',
            "response_schema" : CalendarRequestType
        }        
    )
    result : CalendarRequestType = response.parsed
    logger.info(f"Solicitud enrutada como: {result.request_type} con una confianza de: {result.confidence_score}")
    return result


def handle_new_event(description: str) -> CalendarResponse:
    """Procesa un nuevo evento"""
    logger.info("Procesando una solicitud de un nuevo evento")

    client = genai.Client(api_key=API_KEY)
    system_instructions="Extrae los datos para la creación de un nuevo evento."
    response = client.models.generate_content(
        model=model,
        contents=f"{system_instructions}\n\nUser Query: {description}",
        config = { 
            "response_mime_type" : 'application/json',
            "response_schema" : NewEventDetails
        }        
    )
    details : NewEventDetails = response.parsed
    logger.info(f"Nuevo evento: {details.model_dump_json(indent=2)}")

    # Genera respuesta
    return CalendarResponse(
        success=True,
        message=f"Creado un nuevo evento '{details.name}' para {details.date} con {', '.join(details.participants)}",
        calendar_link=f"calendar://new?event={details.name}",
    )


def handle_modify_event(description: str) -> CalendarResponse:
    """Procesa la modificación de un evento"""
    logger.info("Procesando una solicitud de modificación de evento")

    # Obtener detalles para modificación

    client = genai.Client(api_key=API_KEY)
    system_instructions="Extrae los datos para la modificación de un evento existente."
    response = client.models.generate_content(
        model=model,
        contents=f"{system_instructions}\n\nUser Query: {description}",
        config = { 
            "response_mime_type" : 'application/json',
            "response_schema" : ModifyEventDetails
        }        
    )
    details : ModifyEventDetails = response.parsed
    logger.info(f"Modificar Evento: {details.model_dump_json(indent=2)}")

    # Genera respuesta
    return CalendarResponse(
        success=True,
        message=f"Modificando evento '{details.event_identifier}' añadiendo a {', '.join(details.participants_to_add)} y eliminando a {', '.join(details.participants_to_remove)}",
        calendar_link=f"calendar://new?event={details.event_identifier}",
    )



def process_calendar_request(user_input: str) -> Optional[CalendarResponse]:
    """Main function implementing the routing workflow"""
    logger.info("Procesando solicitud de calendario")

    # Enrutar la solicitud
    route_result = route_calendar_request(user_input)

    # Eluamos el grado de confianza del modelo
    if route_result.confidence_score < 0.7:
        logger.warning(f"Grado de confianza bajo: {route_result.confidence_score}")
        return None

    # Enrutar el handler apropiado
    if route_result.request_type == "nuevo_evento":
        return handle_new_event(route_result.description)
    elif route_result.request_type == "modif_evento":
        return handle_modify_event(route_result.description)
    else:
        logger.warning("Tipo de solicitud no soportada")
        return None


def process_primary_request(user_input: str):
    """Funcion principal para implementar el enrutamiento del trabajo"""
    logger.info("Procesando solicitud primaria")

    # Enrutar la solicitud
    route_result = route_primary_request(user_input)

    # Evaluamos el grado de confianza del modelo
    if route_result.confidence_score < 0.7:
        logger.warning(f"Grado de confianza bajo: {route_result.confidence_score}")
        return None
    
    # Enrutar el handler apropiado
    if route_result.request_type == "calendario":
        return process_calendar_request(route_result.description)
    elif route_result.request_type == "extrac_inf":
        logger.warning("Handler para funcion no implementado")
        return None
    else:
        logger.warning("Tipo de solicitud no soportada")
        return None

# --------------------------------------------------------------
# Step 3: Probar con nuevo evento
# --------------------------------------------------------------

new_event_input = "Programemos una reunión de equipo el próximo martes a las 2 p. m. con Alice y Bob."
result = process_primary_request(new_event_input)
if result:
    print(f"Respuesta: {result.message}")

# --------------------------------------------------------------
# Step 4: Probar con modificar un evento
# --------------------------------------------------------------

modify_event_input = (
    "¿Podrías trasladar la reunión del equipo con Alice y Bob al miércoles a las 3 p. m.?"
)
result = process_primary_request(modify_event_input)
if result:
    print(f"Respuesta: {result.message}")

# --------------------------------------------------------------
# Step 5: Probar con input inválido
# --------------------------------------------------------------

invalid_input = "¿Cómo está el tiempo hoy?"
result = process_primary_request(invalid_input)
if not result:
    print("Solicitud no reconocida como operación de calendario")

2025-03-05 15:02:22 - INFO - Procesando solicitud primaria
2025-03-05 15:02:22 - INFO - Enrutando la solicitud primaria
2025-03-05 15:02:22 - INFO - AFC is enabled with max remote calls: 10.
2025-03-05 15:02:23 - INFO - AFC remote call 1 is done.
2025-03-05 15:02:23 - INFO - Solicitud enrutada como: calendario con una confianza de: 0.95
2025-03-05 15:02:23 - INFO - Procesando solicitud de calendario
2025-03-05 15:02:23 - INFO - Enrutando solicitud de calendario
2025-03-05 15:02:23 - INFO - AFC is enabled with max remote calls: 10.
2025-03-05 15:02:25 - INFO - AFC remote call 1 is done.
2025-03-05 15:02:25 - INFO - Solicitud enrutada como: nuevo_evento con una confianza de: 0.95
2025-03-05 15:02:25 - INFO - Procesando una solicitud de un nuevo evento
2025-03-05 15:02:25 - INFO - AFC is enabled with max remote calls: 10.
2025-03-05 15:02:27 - INFO - AFC remote call 1 is done.
2025-03-05 15:02:27 - INFO - Nuevo evento: {
  "name": "Reunión",
  "date": "2024-03-15T10:00:00",
  "duration_mi

Respuesta: Creado un nuevo evento 'Reunión' para 2024-03-15T10:00:00 con usuario@example.com


2025-03-05 15:02:28 - INFO - AFC remote call 1 is done.
2025-03-05 15:02:28 - INFO - Solicitud enrutada como: calendario con una confianza de: 0.95
2025-03-05 15:02:28 - INFO - Procesando solicitud de calendario
2025-03-05 15:02:28 - INFO - Enrutando solicitud de calendario
2025-03-05 15:02:28 - INFO - AFC is enabled with max remote calls: 10.
2025-03-05 15:02:30 - INFO - AFC remote call 1 is done.
2025-03-05 15:02:30 - INFO - Solicitud enrutada como: modif_evento con una confianza de: 0.95
2025-03-05 15:02:30 - INFO - Procesando una solicitud de modificación de evento
2025-03-05 15:02:30 - INFO - AFC is enabled with max remote calls: 10.
2025-03-05 15:02:32 - INFO - AFC remote call 1 is done.
2025-03-05 15:02:32 - INFO - Modificar Evento: {
  "event_identifier": "team meeting",
  "changes": [
    {
      "field": "date",
      "new_value": "Wednesday"
    },
    {
      "field": "time",
      "new_value": "3 PM"
    }
  ],
  "participants_to_add": [],
  "participants_to_remove": []
}


Respuesta: Modificando evento 'team meeting' añadiendo a  y eliminando a 


2025-03-05 15:02:33 - INFO - AFC remote call 1 is done.
2025-03-05 15:02:33 - INFO - Solicitud enrutada como: extrac_inf con una confianza de: 0.95
2025-03-05 15:02:33 - WARNING - Handler para funcion no implementado


Solicitud no reconocida como operación de calendario
